In [14]:
# !pip install pybaseball
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn
# !pip install matplotlib
# !pip install seaborn
# !pip install pickle5
# !pip install datetime2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [94 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-universal2-cpython-311
      creating build/lib.macosx-10.9-universal2-cpython-311/pickle5
      copying pickle5/__init__.py -> build/lib.macosx-10.9-universal2-cpython-311/pickle5
      copying pickle5/pickletools.py -> build/lib.macosx-10.9-universal2-cpython-311/pickle5
      copying pickle5/pickle.py -> build/lib.macosx-10.9-universal2-cpython-311/pickle5
      creating build/lib.macosx-10.9-universal2-cpython-311/pickle5/test
      copying pickle5/test/test_pickle.py -> build/lib.macosx-10.9-univers

In [15]:
# Start by importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

from pybaseball import schedule_and_record
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import team_batting
from pybaseball import team_pitching
from pybaseball import batting_stats
from pybaseball import pitching_stats
from pybaseball import batting_stats_range
from pybaseball import pitching_stats_range
from pybaseball import statcast_single_game
from pybaseball import playerid_reverse_lookup

In [16]:
# fangraphs data pull
pitching_df = pitching_stats(2021, 2022)
batting_df = batting_stats(2021, 2022)

In [19]:
print(batting_df.shape)
print(pitching_df.shape)

(262, 320)
(84, 334)


In [20]:
pitching_df

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
6,19361,2021,Corbin Burnes,MIL,26,11,5,7.5,2.43,28,...,8.4,12,0.031,114.2,117,0.305,383,0.172,0.338,NaN
14,10310,2021,Zack Wheeler,PHI,31,14,10,7.2,2.78,32,...,8.4,25,0.046,118.6,156,0.285,548,0.156,0.280,NaN
38,16149,2022,Aaron Nola,PHI,29,11,13,6.3,3.25,32,...,12.5,38,0.071,112.0,169,0.316,534,0.197,0.324,NaN
19,16137,2022,Carlos Rodon,SFG,29,14,8,6.2,2.88,31,...,19.4,27,0.065,112.5,166,0.397,418,0.164,0.306,NaN
0,8700,2022,Justin Verlander,HOU,39,18,4,6.1,1.75,28,...,16.9,28,0.063,110.9,155,0.348,446,0.150,0.265,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,14168,2022,Jose Berrios,TOR,28,12,7,1.1,5.23,32,...,13.9,52,0.095,113.8,238,0.434,548,0.184,0.277,NaN
82,9434,2021,Dallas Keuchel,CHW,33,9,9,0.7,5.28,32,...,5.0,50,0.089,115.6,222,0.397,559,0.167,0.255,NaN
83,9323,2021,Patrick Corbin,WSN,31,9,16,0.2,5.82,31,...,9.0,50,0.092,112.9,222,0.407,545,0.153,0.265,NaN
64,15467,2022,Marco Gonzales,SEA,30,10,15,0.1,4.13,32,...,14.5,45,0.072,111.2,215,0.345,623,0.151,0.235,NaN


In [18]:
batting_df.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246,0.609,404,0.169,0.287,NaN,NaN,NaN,11.2
18,11493,2022,Manny Machado,SDP,29,150,578,644,172,102,...,112.4,219,0.490,447,0.126,0.243,NaN,NaN,NaN,6.6
6,19709,2021,Fernando Tatis Jr.,SDP,22,130,478,546,135,62,...,116.6,183,0.556,329,0.102,0.270,NaN,NaN,NaN,6.2
19,9777,2022,Nolan Arenado,STL,31,148,557,620,163,90,...,111.4,190,0.389,489,0.155,0.241,NaN,NaN,NaN,7.2
4,9218,2022,Paul Goldschmidt,STL,34,151,561,651,178,102,...,112.3,200,0.469,426,0.196,0.295,NaN,NaN,NaN,8.0


In [20]:
data2 = schedule_and_record(2022, 'MIL')

In [25]:
data2

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
1,"Thursday, Apr 7",MIL,@,CHC,L,4.0,5.0,9.0,0-1,4.0,1.0,Givens,Ashby,Robertson,3:18,D,35112.0,.99,-1.0,None
2,"Saturday, Apr 9",MIL,@,CHC,L,0.0,9.0,9.0,0-2,4.0,2.0,Steele,Woodruff,None,3:28,D,30369.0,.99,-2.0,None
3,"Sunday, Apr 10",MIL,@,CHC,W,5.0,4.0,9.0,1-2,4.0,1.0,Cousins,Norris,Hader,3:03,D,32858.0,.93,1.0,None
4,"Monday, Apr 11",MIL,@,BAL,L,0.0,2.0,9.0,1-3,5.0,1.5,Baumann,Houser,Lopez,3:24,D,44461.0,.89,-1.0,None
5,"Tuesday, Apr 12",MIL,@,BAL,W,5.0,4.0,9.0,2-3,3.0,1.5,Milner,Bautista,Hader,3:38,N,11814.0,.87,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,"Saturday, Oct 1",MIL,Home,MIA,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None
159,"Sunday, Oct 2",MIL,Home,MIA,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None
160,"Monday, Oct 3",MIL,Home,ARI,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None
161,"Tuesday, Oct 4",MIL,Home,ARI,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,NaN,None,NaN,None


In [29]:
data3.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
3,19709,2021,Fernando Tatis Jr.,SDP,22,130,478,546,135,62,...,0.213,116.6,183,0.556,329,0.102,0.270,0.279,0.618,0.406
1,20123,2021,Juan Soto,WSN,22,151,502,654,157,106,...,0.133,116.6,217,0.524,414,0.200,0.263,0.304,0.544,0.430
8,16252,2021,Trea Turner,- - -,28,148,595,646,195,130,...,0.074,112.2,225,0.460,489,0.160,0.262,0.303,0.484,0.362
9,19326,2021,Bryan Reynolds,PIT,26,159,559,646,169,102,...,0.104,112.7,181,0.408,444,0.123,0.231,0.293,0.522,0.385
0,11579,2021,Bryce Harper,PHI,28,141,488,599,151,73,...,0.181,116.3,177,0.492,360,0.119,0.263,0.301,0.610,0.430
